In [1]:
from ActorCritic import ActorCritic
from ActorCritic import *
from utils import *
from derivative_routines import *
import numpy as np
import time as t

Using TensorFlow backend.


In [2]:
# generate all the data
state_dim = 3
action_dim = 2
param_dim = 5
mb_size = 128
state = np.random.normal(loc=5.0, size=(mb_size,state_dim)).reshape(mb_size,state_dim)
action = np.random.normal(loc=2.0, size=(mb_size,action_dim)).reshape(mb_size,action_dim)
reward = np.random.normal(loc=1.0, size=(mb_size,1)).reshape(mb_size,1)
next_state = np.random.normal(loc=5.0, size=(mb_size,state_dim)).reshape(mb_size,state_dim)
params = np.random.normal(loc=10.0, size=(mb_size, param_dim)).reshape(mb_size,param_dim)

In [3]:
agent = ActorCritic(state_dim, action_dim, param_dim)

In [4]:
# push the above generated data to ER
for i in range(mb_size):
    agent.append_memory(state[i], action[i], reward[i], next_state[i], params[i], 0)

In [5]:
wts_b4 = unpack_theta(agent.learner.net1)
t1 = t.time()
for _ in range(500):
    agent.update_learners()
wts_af1 = unpack_theta(agent.learner.net1)
t2 = t.time()
print "For 500 learner updates, time taken=", t2-t1

For 500 learner updates, time taken= 5.13072085381


In [6]:
# The above is different from 500 epochs with same minibatch samples
t1 = t.time()
agent.update_learners(epochs=500)
t2 = t.time()
print "Time for 500 epochs = ", t2-t1

Time for 500 epochs =  2.57240200043


- Bottle neck is really subsampling the replay memory to prepare data
- One way to speed up implimentation: update network once in T time steps, but use T epochs (say T=25)
- Within T steps the distribution of ER is unlikely to change much, so the gradient steps won't differ much either

In [12]:
# How fast is the target network updates?
t1 = t.time()
for _ in range(100):
    agent.update_target_networks()
t2 = t.time()
print "For 100 target network updates, time taken=", t2-t1

For 100 target network updates, time taken= 32.3734028339


The bottle neck is really in the set_model_params() function. There is no workaround this -- I need to extract a proper numpy array from the model (this is super fast) and then set the weights back in the appropriate place (super slow).   ---> speeding this part up helps